# 1. Imports & Environment Setup

- **Core Imports:**
  - `os`, `warnings`, `datetime`, `uuid`, `string`, `random`, `hashlib`, `traceback`.

- **LangChain/LangGraph Imports:**
  - `ChatOpenAI`, `StreamingStdOutCallbackHandler`, `TavilySearch`, `TavilySearchResults`, agent/graph/integration tools, message types.

- **Notebook & Display:**
  - `clear_output`, `Markdown` from `IPython.display`.

- **Environment:**
  - Suppresses TensorFlow warnings.
  - Loads environment variables (e.g., API keys) using `python-dotenv`.


In [1]:
import os
import warnings
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_tavily import TavilySearch
from langchain_community.tools import TavilySearchResults
from langchain.agents import create_tool_calling_agent, AgentExecutor
from typing import TypedDict, Dict, List
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from datetime import datetime
from IPython.display import clear_output, Markdown
import traceback
import uuid
import string
import random
import hashlib

In [2]:

# Suppress TensorFlow warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
warnings.filterwarnings("ignore")

# Load environment variables
load_dotenv()

def get_api_key(key_name="OPENROUTER_API_KEY"):
    """
    Get API key from environment variables
    """
    api_key = os.getenv(key_name)
    if not api_key:
        raise ValueError(f"Invalid API key: {key_name} not found in environment variables")
    return api_key

def initialize_llm(model_name="meta-llama/llama-3.1-8b-instruct",
                  temperature=0.4,
                  use_streaming=True):
    """
    Initialize LLM
    """
    api_key = get_api_key()
    callbacks = [StreamingStdOutCallbackHandler()]
    llm = ChatOpenAI(
        model_name=model_name,
        temperature=temperature,
        streaming=use_streaming,
        callbacks=callbacks,
        openai_api_key=api_key,
        openai_api_base="https://openrouter.ai/api/v1"
    )
    return llm

llm = initialize_llm()

python-dotenv could not parse statement starting at line 12


In [3]:
def save_file(data, filename, uniqueness_method="uuid"):

    folder_name = "Travel Plans"  # Folder to store output files
    os.makedirs(folder_name, exist_ok=True)  # Creates the folder if it doesn't exist
    
    # Generate unique identifier based on selected method
    if uniqueness_method == "uuid":
        # Generate a UUID (universally unique identifier)
        unique_id = str(uuid.uuid4())[:8]
        
    elif uniqueness_method == "random_string":
        # Generate a random string of letters and digits
        chars = string.ascii_letters + string.digits
        unique_id = ''.join(random.choice(chars) for _ in range(8))
        
    elif uniqueness_method == "hash":
        # Create a hash based on content and current time
        content_hash = hashlib.md5((data + str(datetime.now())).encode()).hexdigest()[:8]
        unique_id = content_hash
        
    elif uniqueness_method == "counter":
        # Use an incrementing counter for files with the same base name
        counter = 1
        while True:
            test_filename = f"{filename}_{counter}.md"
            test_path = os.path.join(folder_name, test_filename)
            if not os.path.exists(test_path):
                unique_id = str(counter)
                break
            counter += 1
            
    elif uniqueness_method == "datetime":
        # Original datetime method
        unique_id = datetime.now().strftime("%Y%m%d%H%M%S")
        
    else:
        # Default to UUID if an invalid method is specified
        unique_id = str(uuid.uuid4())[:8]
    
    # Create the final filename with the unique identifier
    final_filename = f"{filename}_{unique_id}.md"
    file_path = os.path.join(folder_name, final_filename)
    
    # Save the data to the file in the specified path
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(data)
        print(f"File '{file_path}' created successfully.")
    
    # Return the full path of the saved file
    return file_path

def show_md_file(file_path):
    with open(file_path, "r", encoding = "utf-8") as file:
        content = file.read()

    display(Markdown(content))

In [4]:
class PlannerState(TypedDict):
    travel_preferences: Dict
    destination_options: List[Dict]
    selected_destination: Dict
    budget_plan: Dict
    itinerary: List[Dict]
    bookings: Dict
    feedback: Dict
    final_trip: Dict
    messages: List
    status: str

In [5]:
def input_collector(state: PlannerState) -> PlannerState:
    """Collects and organizes user's travel preferences."""
    messages = state["messages"]
    
    # Extract the latest human message
    human_message = next((m for m in reversed(messages) if isinstance(m, HumanMessage)), None)
    system_prompt = """You are a travel input collector. Extract travel preferences from the user's message.
    Include: destination interests, travel dates, budget range, number of travelers, 
    accommodation preferences, activity interests, and any special requirements.
    """
    
    response = llm.invoke([
        SystemMessage(content = system_prompt),
        human_message
    ])
    
    # Parse the response into structured travel preferences
    preferences = {
        "raw_input": human_message.content,
        "processed_preferences": response.content,
        "timestamp": datetime.now().isoformat()
    }
    
    # Update state
    state["travel_preferences"] = preferences
    state["messages"].append(AIMessage(content=f"I've collected your travel preferences. Moving on to destination research."))
    state["status"] = "preferences_collected"
    
    return state

In [6]:
def destination_research(state: PlannerState) -> PlannerState:
    """Researches and suggests destinations based on user preferences"""
    preferences = state["travel_preferences"]
    system_prompt = """
    You are a destination research expert. Based on the user's travel preferences, suggest 
    3-5 suitable destinations. For each destination, provide:
    - Name and brief description
    - Why it matches their preferences
    - Best time to visit
    - Estimated overall cost level
    - Key attractions
    """
    
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content = f"Travel Preferences: {preferences}")
    ])
    
    # Process the response into structured destination options
    destination_options = {
        "options": response.content,
        "research_timestamp": datetime.now().isoformat()
    }
    
    # Update State
    state["destination_options"] = destination_options
    state["messages"].append(AIMessage(content=f"I've researched some destinations that match your preferences. Moving on to budget planning."))
    state["status"] = "awaiting_destination_selection"
    
    return state

In [7]:
def collect_selected_destination(state: PlannerState) -> PlannerState:
    """Captures user's selected destination from suggestions"""
    messages = state["messages"]
    
    latest_human = next((m for m in reversed(messages) if isinstance(m, HumanMessage)), None)
    
    if not latest_human:
        raise ValueError("No user input found for destination selection.")

    selected_destination = {
        "name": latest_human.content,
        "details": "To be looked up or parsed from previous suggestions."
    }

    state["selected_destination"] = selected_destination
    state["messages"].append(AIMessage(content=f"Thanks! You've selected **{latest_human.content}**. We'll proceed to budget planning."))
    state["status"] = "destination_selected"
    
    return state

In [8]:
def budget_planner(state: PlannerState) -> PlannerState:
    preferences = state["travel_preferences"]
    destination = state["selected_destination"]
    
    system_prompt = """
    You are a travel budget planner. Create a detailed budget breakdown for the trip, including:
    - Transportation (flights, local transit)
    - Accommodation costs
    - Daily expenses (food, activities)
    - Recommended spending money
    - Potential extras and contingencies
    """
    
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Travel preferences: {preferences}, Selected destination: {destination}")
    ])
    
    # Process the response into a structured budget plan
    budget_plan = {
        "breakdown": response.content,
        "timestamp": datetime.now().isoformat()
    }
    
    # Update state
    state["budget_plan"] = budget_plan
    state["messages"].append(AIMessage(content=f"I've created a budget plan for your trip. Now building your itinerary."))
    state["status"] = "budget_planned"
    
    return state

In [9]:
def itinerary_builder(state: PlannerState) -> PlannerState:
    preferences = state["travel_preferences"]
    destination = state["selected_destination"]
    budget = state["budget_plan"]
    
    system_prompt = """
    You are a travel itinerary expert. Create a detailed day-by-day itinerary for the trip, including:
    - Arrival and departure logistics
    - Daily activities and attractions
    - Meal recommendations
    - Rest periods and flexibility
    - Special experiences or highlights
    Balance the itinerary to avoid exhaustion while maximizing experiences within budget constraints.
    """
    
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Travel preferences: {preferences}, Selected destination: {destination}, Budget plan: {budget}")
    ])
    
    # Process the response into a structured itinerary
    itinerary = {
        "daily_plan": response.content,
        "timestamp": datetime.now().isoformat()
    }
    
    # Update state
    state["itinerary"] = itinerary
    state["messages"].append(AIMessage(content=f"Your itinerary is ready! Moving on to booking assistance."))
    state["status"] = "itinerary_built"
    
    return state

In [10]:
def booking_assistant(state: PlannerState) -> PlannerState:
    """Provides booking recommendations and assistance."""
    preferences = state["travel_preferences"]
    destination = state["selected_destination"]
    itinerary = state["itinerary"]
    
    system_prompt = """
    You are a travel booking assistant. Provide detailed booking recommendations, including:
    - Flight options and booking strategies
    - Accommodation recommendations with booking links/platforms
    - Activity reservation requirements
    - Transportation booking needs
    - Travel insurance options
    Include tips for getting the best deals and necessary booking timelines.
    """
    
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"Travel preferences: {preferences}, Selected destination: {destination}, Itinerary: {itinerary}")
    ])
    
    # Process the response into structured booking information
    bookings = {
        "recommendations": response.content,
        "timestamp": datetime.now().isoformat()
    }
    
    # Update state
    state["bookings"] = bookings
    state["messages"].append(AIMessage(content=f"Here are your booking recommendations. Please provide feedback on the plan."))
    state["status"] = "bookings_provided"
    
    return state

In [11]:
def trip_exporter(state: PlannerState) -> PlannerState:
    """Creates a final trip package with all details."""
    preferences = state["travel_preferences"]
    destination = state["selected_destination"]
    budget = state["budget_plan"]
    itinerary = state["itinerary"]
    bookings = state["bookings"]
    
    system_prompt = """
    You are a travel document preparation specialist. Create a comprehensive trip package including:
    - Trip summary with key details
    - Complete itinerary with timings
    - Budget breakdown
    - Booking details and confirmation needs
    - Packing recommendations
    - Travel tips specific to the destination
    - Emergency contacts and resources
    Format this as a complete travel document that the traveler can use throughout their journey.
    """
    
    response = llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=f"""
        Travel preferences: {preferences}
        Selected destination: {destination}
        Budget plan: {budget}
        Itinerary: {itinerary}
        Booking details: {bookings}
        """)
    ])
    
    # Process the response into a final trip package
    final_trip = {
        "package": response.content,
        "created_at": datetime.now().isoformat()
    }
    
    # Update state
    state["final_trip"] = final_trip
    state["messages"].append(AIMessage(content=f"Your trip package is ready! Here's everything you need for your journey."))
    state["status"] = "trip_exported"
    
    return state

# Add nodes

In [12]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(PlannerState)

workflow.add_node("input_collector", input_collector)
workflow.add_node("destination_research", destination_research)
workflow.add_node("collect_selected_destination", collect_selected_destination)
workflow.add_node("budget_planner", budget_planner)
workflow.add_node("itinerary_builder", itinerary_builder)
workflow.add_node("booking_assistant", booking_assistant)
workflow.add_node("trip_exporter", trip_exporter)

# Add Edges
workflow.set_entry_point("input_collector")
workflow.add_edge("input_collector", "destination_research")
workflow.add_edge("destination_research", "collect_selected_destination")
workflow.add_edge("collect_selected_destination", "budget_planner")
workflow.add_edge("budget_planner", "itinerary_builder")
workflow.add_edge("itinerary_builder", "booking_assistant")
workflow.add_edge("booking_assistant", "trip_exporter")
workflow.add_edge("trip_exporter", END)

graph = workflow.compile()

In [13]:
# Assuming PlannerState and graph are defined elsewhere
# Initialize state globally or pass it as a parameter
state = PlannerState(
    travel_preferences={},
    destination_options=[],
    selected_destination={},
    budget_plan={},
    itinerary=[],
    bookings={},
    feedback={},
    final_trip={},
    messages=[],
    status="initial"
)

def process_travel_query(query, planner_state):
    """
    Process a single user query for the travel planning workflow.
    
    Args:
        query (str): The user's input query.
        planner_state (PlannerState): The current state of the planner.
    
    Returns:
        tuple: (response, updated_state)
            - response (str): The assistant's response or error message.
            - updated_state (PlannerState): The updated state after processing.
    """
    # Exit condition
    if query.lower() == "exit":
        return "Exiting the travel planner.", planner_state

    # Add user input to the state
    if query.lower() != "continue":
        planner_state["messages"].append(HumanMessage(content=query))

    # Debug: Print current state status and messages
    print(f"\n[Debug] Current status: {planner_state['status']}")
    print(f"[Debug] Current messages: {[msg.content for msg in planner_state['messages']]}")

    # Process the workflow step-by-step
    try:
        assistant_response = ""
        # Stream through the workflow
        for output in graph.stream(planner_state):
            for node, updated_state in output.items():
                print(f"\n[Node: {node}]")
                # Collect new assistant messages
                for message in updated_state["messages"]:
                    if isinstance(message, AIMessage) and message not in planner_state["messages"]:
                        assistant_response += f"{message.content}\n"
                planner_state = updated_state

                # Debug: Print updated state status
                print(f"[Debug] Updated status after {node}: {planner_state['status']}")

                # Check if workflow is complete
                if planner_state["status"] == "trip_exported":
                    final_package = planner_state["final_trip"].get("package", "No package available.")
                    assistant_response += f"\nFinal Trip Package:\n{final_package}\n\nPlanning complete!"
                    return assistant_response, planner_state

        # If no assistant response was generated, return a prompt based on the current status
        if not assistant_response:
            if planner_state["status"] == "initial":
                assistant_response = "Enter your travel preferences (e.g., destination, dates, budget, etc.): "
            elif planner_state["status"] == "awaiting_destination_selection":
                options = planner_state.get("destination_options", {}).get("options", "No options available yet.")
                assistant_response = f"\nSuggested destinations:\n{options}\nPlease select a destination from the suggested options: "
            else:
                assistant_response = "Provide any additional input or feedback (or type 'continue' to proceed): "

        return assistant_response, planner_state

    except Exception as e:
        error_message = f"Error occurred: {e}\nStack trace:\n{traceback.format_exc()}\nPlease check your setup (e.g., API keys, dependencies) and try again."
        return error_message, planner_state



# Simulate a few queries
queries = [
    "I want to travel to Kenya in June 2025 with a budget of $2000. It should be a family trip. We love water activities and safaris",
    "continue",
    "Nairobi sounds good, let's select it.",
    "exit"
]

print("Welcome to the Travel Planning Assistant!")
current_state = state  # Use the initialized state
for query in queries:
    response, current_state = process_travel_query(query, current_state)
    print(f"\nAssistant: {response}")

    # Save and show response if workflow is complete
    if query.lower() == "exit" or current_state["status"] == "trip_exported":
        file_path = save_file(response, filename="travel_plan", uniqueness_method="datetime")
        clear_output()
        show_md_file(file_path)
        break


Final Trip Package:
**Kenya Family Trip Travel Document**

**Trip Summary**

* Destination: Kenya
* Travel Dates: June 2025
* Budget: $2000
* Number of Travelers: Family (3 or more)
* Accommodation Preferences: Family-friendly hotel or resort
* Activity Interests: Water activities, safaris

**Itinerary**

**Day 1: Arrival in Nairobi and Transfer to Lake Nakuru**

* Morning: Arrive at Jomo Kenyatta International Airport in Nairobi
* Transfer to Lake Nakuru Lodge (approx. $20-50)
* Check-in at Lake Nakuru Lodge, a family-friendly hotel with stunning views of the lake (approx. $100-150 per night)
* Afternoon: Visit the nearby Lake Nakuru National Park for a scenic drive and birdwatching (approx. $20-50 per person)
* Evening: Enjoy dinner at the lodge's restaurant and relax by the lake

**Day 2: Lake Nakuru National Park**

* Morning: Take a guided boat tour on Lake Nakuru (approx. $50-100 per person)
* Afternoon: Visit the Lake Nakuru National Park for a safari drive (approx. $100-200 per person)
* Evening: Enjoy dinner at a local restaurant in the nearby town of Nakuru

**Day 3: Transfer to Maasai Mara National Reserve**

* Morning: Transfer to the Maasai Mara National Reserve (approx. $100-200)
* Check-in at a safari lodge in the Maasai Mara (approx. $100-150 per night)
* Afternoon: Take a guided safari drive in the Maasai Mara National Reserve (approx. $100-200 per person)
* Evening: Enjoy dinner at the lodge's restaurant and relax around the campfire

**Day 4: Maasai Mara National Reserve**

* Morning: Take a hot air balloon ride over the Maasai Mara National Reserve (approx. $200-300 per person)
* Afternoon: Visit a local Maasai village for a cultural experience (approx. $20-50 per person)
* Evening: Enjoy dinner at a local restaurant in the nearby town of Narok

**Day 5: Maasai Mara National Reserve**

* Morning: Take a guided safari drive in the Maasai Mara National Reserve (approx. $100-200 per person)
* Afternoon: Visit a nearby waterhole for a relaxing swim or kayak experience (approx. $20-50 per person)
* Evening: Enjoy dinner at the lodge's restaurant and relax by the campfire

**Day 6: Departure from Nairobi**

* Morning: Transfer back to Nairobi (approx. $100-200)
* Depart from Jomo Kenyatta International Airport for your return journey

**Budget Breakdown**

* Transportation: $920-1,450 (approx. $154-242 per person)
* Accommodation: $800-1,200 (approx. $133-200 per person)
* Daily Expenses:
	+ Food: $300-600 (approx. $50-100 per person per day)
	+ Activities: $500-1,000 (approx. $83-167 per person per day)
	+ Miscellaneous: $200-400 (approx. $33-67 per person)
* Total: $2,020-3,750 (approx. $335-625 per person)

**Booking Details**

* Flights: Book flights to Nairobi in June 2025 as soon as possible to get the best deals. Consider flying into Nairobi's Jomo Kenyatta International Airport (NBO) and out of Nairobi to save on transportation costs.
* Accommodations:
	+ Lake Nakuru Lodge: Book a family-friendly room at Lake Nakuru Lodge, which offers stunning views of the lake and is a great base for exploring the nearby national park.
	+ Maasai Mara Safari Lodge: Consider booking a safari lodge that offers family-friendly accommodations and guided safari drives.
* Activities:
	+ Lake Nakuru National Park: Book a guided boat tour and safari drive in advance through the park's official website or a reputable tour operator like Abercrombie & Kent.
	+ Maasai Mara National Reserve: Book a guided safari drive and hot air balloon ride through a reputable tour operator like Abercrombie & Kent or Governors' Camp.
	+ Maasai Village Visit: Book a cultural experience through a reputable tour operator like Abercrombie & Kent or Governors' Camp.
* Transportation: Book a private transfer from the airport to Lake Nakuru Lodge through a reputable company like Nairobi Shuttle Services or Airport Shuttle Services.
* Travel Insurance: Consider purchasing travel insurance that covers trip cancellations, interruptions, and medical emergencies.

**Packing Recommendations**

* Pack light and comfortable clothing for the safari drives and water activities.
* Bring sunscreen, sunglasses, and a hat for outdoor protection.
* Respect local cultures and wildlife during the safari drives and cultural experiences.

**Travel Tips**

* Book flights, accommodations, and activities at least 2-3 months in advance to secure the best prices.
* Compare prices using flight comparison websites and online travel agencies.
* Consider a group tour or package deal that includes transportation, accommodation, and activities to save money.
* Pack smart and respect local cultures and wildlife.

**Emergency Contacts and Resources**

* Embassy of the United States in Nairobi: +254 20 363 6000
* Kenya Tourism Board: +254 20 222 2222
* Abercrombie & Kent: +254 20 363 6000
* Governors' Camp: +254 20 363 6000

**Confirmation Needs**

* Confirm flight bookings at least 2-3 months in advance.
* Confirm accommodation bookings at least 2-3 months in advance.
* Confirm activity bookings at least 2-3 months in advance.

**Booking Timelines**

* Flights: Book flights by June 2024 to secure the best prices.
* Accommodations: Book accommodations by July 2024 to ensure availability.
* Activities: Book activities and tours by August 2024 to secure the best prices.

Planning complete!